In [25]:
import os
from dotenv import load_dotenv
load_dotenv() 

True

In [26]:
os.getenv("LANGCHAIN_PROJECT")

'AgenticAI'

In [27]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

## This value has to be set to true always
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "o1-mini")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x156f929e0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x156f92ea0> root_client=<openai.OpenAI object at 0x156f90180> root_async_client=<openai.AsyncOpenAI object at 0x156f92b10> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [29]:
result = llm.invoke("What is the capital of India?")
result

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 14, 'total_tokens': 99, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'o1-mini-2024-09-12', 'system_fingerprint': 'fp_3da8b0b088', 'id': 'chatcmpl-BcmmMLCz9In5AYnDo439MFfbrQmlE', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--af5b0413-5bb6-4db3-8b37-1ef5dce65351-0', usage_metadata={'input_tokens': 14, 'output_tokens': 85, 'total_tokens': 99, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [30]:
result.content

'The capital of India is **New Delhi**.'

In [31]:
from langchain_groq import ChatGroq

model = ChatGroq(model="qwen-qwq-32b")

In [32]:
model.invoke("Hi, I am Rhythm")

AIMessage(content='\n<think>\nOkay, the user said "Hi, I am Rhythm." I need to respond in a friendly way. Let me start by greeting them back. Maybe say "Hello, Rhythm!" to acknowledge their name. Then, I should offer assistance. I can ask how I can help them today. Keep it simple and open-ended so they feel comfortable to ask anything. Let me check if there\'s anything else I should consider. No, that\'s probably enough. Alright, let\'s put it together.\n\nWait, maybe add an emoji to make it more welcoming? Like a smiley face? The user might appreciate a friendly tone. Yeah, that\'s good. So the response would be: "Hello, Rhythm! 😊 How can I assist you today?" That sounds good. Short, friendly, and invites them to state their request. I think that\'s all.\n</think>\n\nHello, Rhythm! 😊 How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 193, 'prompt_tokens': 16, 'total_tokens': 209, 'completion_time': 0.474020867, 'prompt_time': 0.

### PROMPT ENGINEERING

In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt =  ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert AI assistant. Provide me with concise and accurate answers."),
        ("user", "{input}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI assistant. Provide me with concise and accurate answers.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [44]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="gemma2-9b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x110161bf0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x110162030>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [45]:
chain = prompt | llm

chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI assistant. Provide me with concise and accurate answers.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x110161bf0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x110162030>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [46]:
response = chain.invoke({"input": "Can you tell me about the history of the Eiffel Tower?"})
response.content

"The Eiffel Tower was built as the centerpiece of the 1889 World's Fair in Paris, celebrating the centennial of the French Revolution. \n\n**Key points:**\n\n* **Designed by Gustave Eiffel:** A renowned engineer, his company specialized in wrought-iron structures.\n* **Construction:** Took 2 years, 2 months, and 5 days, from 1887 to 1889.\n* **Initial Controversy:** Many Parisians opposed its construction, considering it an eyesore.\n* **Intended as Temporary:**  Initially planned to be dismantled after 20 years, it was saved due to its use for radio transmission.\n* **Iconic Landmark:** It became a symbol of Paris and France, attracting millions of visitors annually.\n\nThe Eiffel Tower has undergone several renovations and upgrades over the years, solidifying its status as a global icon.\n"

### Output Parser

In [47]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [48]:
response = chain.invoke({"input": "What is the capital of France?"})
print(response)

Paris.



In [52]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

output_parser = JsonOutputParser()

prompt = PromptTemplate(template = "Answer the user query \n {format_instructions} \n {input} \n", input_variables = ["query"], 
                        partial_variables = {"format_instructions": output_parser.get_format_instructions()})

chain = prompt | llm | output_parser

In [53]:
response = chain.invoke({"input": "What is Bitcoin in the world of Finance?"})
print(response)

{'description': 'Bitcoin is a decentralized digital currency, without a central bank or single administrator, that can be sent from user to user on the peer-to-peer bitcoin network without the need for intermediaries.', 'key_features': ['Decentralized:', 'Cryptocurrency:', 'Limited Supply:', 'Pseudonymous:', 'Secure Transactions:'], 'impact_on_finance': ['Alternative Investment Asset:', 'Potential for Disruption:', 'Increased Financial Inclusion:', 'New Payment Options:'], 'risks': ['Volatility:', 'Security Risks:', 'Regulatory Uncertainty:']}
